In [1719]:
# -*- coding:utf-8 -*-
import pandas as pd
import random
import re
import numpy as np
from zhon.hanzi import punctuation
from numpy import * 
from sklearn.model_selection import train_test_split
from math import *
from pythonds.trees import binaryTree
from treelib import Node, Tree
import copy

In [1720]:
# 1. 读文件，存到List中

def readFile(file):
    df = pd.read_excel(file,encoding ="utf-8")
    sl=df['sepal_length'].values.tolist()
    sw=df['sepal_width'].values.tolist()
    fl=df['flower_length'].values.tolist()
    fw=df['flower_width'].values.tolist()
    c=df['classify'].values.tolist()
    
    ori_set = []
    for a,b,c,d,e in zip(sl,sw,fl,fw,c):
        tmp = [a,b,c,d,e]
        ori_set.append(tmp)
    
    return ori_set

In [1721]:
ori_set = readFile('./Iris.xlsx')

In [1722]:
# 2. 划分数据集：训练集和测试集

def divide_data(ori_set):
    train,test = train_test_split(ori_set,test_size=0.2,random_state=1)
    
    return train, test

In [1723]:
train, test = divide_data(ori_set)
train_copy = train

In [1724]:
# 3. 计算根节点信息熵

def rootE(train):
    setosa_no = 0
    versicolor_no = 0
    virginica_no = 0
    for item in train:
        if item[4] == 'Iris-setosa':          #'Iris-setosa'样本个数
            setosa_no += 1
        if item[4] == 'Iris-versicolor':          #'Iris-versicolor'样本个数
            versicolor_no += 1
        if item[4] == 'Iris-virginica':           #'Iris-virginica'样本个数 
            virginica_no += 1
    total_no = setosa_no + versicolor_no + virginica_no
    ent = -1 * (setosa_no/total_no * log(setosa_no/total_no,2) + 
                versicolor_no/total_no * log(versicolor_no/total_no,2) + 
                virginica_no/total_no * log(virginica_no/total_no,2))

    return ent

In [1725]:
root_ent = rootE(train)

In [1726]:
# 计算熵通式
def ent(type1, type2, type3, total):
    if type1!=0 and type2 !=0 and type3!=0:
        Hs = -1 * (type1/total * log(type1/total,2) + 
                   type2/total * log(type2/total,2) + 
                   type3/total * log(type3/total,2) )
    else:
        if type1 ==0:
            k1 = 0
        else:
            k1 = type1/total * log(type1/total,2)
        if type2 ==0:
            k2 = 0
        else:
            k2 = type2/total * log(type2/total,2)
        if type3 ==0:
            k3 = 0
        else:
            k3 = type3/total * log(type3/total,2)
        Hs = -1 * (k1 + k2 + k3)
        
    if Hs == -0.0:
        Hs = 0
    
    return Hs

In [1727]:
# 数每个属性对应的分类的个数 D-、D+

def count_type(train, value):
    minD = [0,0,0]
    maxD = [0,0,0]
    for t in train:
        # D(-)
        if t[0] <= value:          
            if t[-1] == 'Iris-setosa':
                minD[0] += 1
            if t[-1] == 'Iris-versicolor':
                minD[1] += 1
            if t[-1] == 'Iris-virginica':
                minD[2] += 1
        # D(+)
        else:
            if t[-1] == 'Iris-setosa':
                maxD[0] += 1
            if t[-1] == 'Iris-versicolor':
                maxD[1] += 1
            if t[-1] == 'Iris-virginica':
                maxD[2] += 1
        
    return minD, maxD

In [1728]:
# 输入：（某一属性数组，父亲的Ent，所有数据集数组train）
# 输出：某一属性此时的gain

def gain(attr, parent_ent, train):
    
    #attr从小到大排序
    attr.sort()          
    
    # T为候选划分点集合，个数为 N-1
    T = []
    i = 0
    for i in range(len(attr) - 1):
        value = (attr[i] + attr[i+1]) / 2
        T.append(value)
        
    # 分别计算Gain值    
    index = 1
    gain_list = []
    for t in T:
        #if t 
        minD, maxD = count_type(train, t)          # minD=[1,2,3] (D-) ,  maxD=[1,2,3] (D+)
        ED_min = ent(minD[0], minD[1], minD[2], index)
        ED_plus = ent(maxD[0], maxD[1], maxD[2], len(train)-index )
        Gain = parent_ent - (ED_min * index/ len(train) + ED_plus * (len(train)-index)/len(train) )
        gain_list.append(Gain)
        index += 1
    # 选取Gain最大值作为划分点
    p_index = gain_list.index(max(gain_list))  
    result_gain = max(gain_list)
    division_p = T[p_index]                         #划分点
    
    return result_gain, division_p

In [1729]:
# 选择父节点
def choose_parent(train, num):
    
    # 4. 计算每个属性的信息增益   
    maxGainChosing = []
    dividing_point = []
    for index in range(num):
        tmp = []
        for elem in train:
            tmp.append(elem[index])
        
        result_gain, division_p = gain(tmp, root_ent, train)           ####
        maxGainChosing.append(result_gain)
        dividing_point.append(division_p)
    
    # 5. 选出信息增益最大的属性index，作为划分属性
    rootGain = max(maxGainChosing)
    rootIndex = maxGainChosing.index(max(maxGainChosing)) 
    division_p = dividing_point[rootIndex]
    #print('Root[',rootIndex,']:',rootGain, ', t = ',division_p)
    
    return rootIndex,division_p

In [1730]:
# 获取该划分元素名字

def getPointName(index, turn, labels):
    if turn == 1:                       #第一轮
        labels = ['sepal_length', 'sepal_width', 'flower_length', 'flower_width']
        label = labels[index]
        del labels[index]
    
    else:                               #非第一轮
        label = labels[index]
        del labels[index]
        
    return labels, label
        

In [1731]:
# 更新train
def update_train(train, index, division_p):
    subset1 = []
    subset2 = []
    
    
    #数据集再划分
    for t in train:
        if t[index]<= division_p:                  # t[index] <= 划分点
            subset1.append(t)
        else:                                      # t[index] > 划分点
            subset2.append(t)
            
    #删除划分属性       
    for s1,s2 in zip(subset1,subset2):
        del s1[index]
        del s2[index]
    
    return subset1, subset2

In [1732]:
# 判断是否可以再分：若不可分，返回label种类；若可分，

def dividable(dataSet, labels):
    classList = [example[-1] for example in dataSet]         # ['A', 'B', 'C', 'A', 'B', 'C', 'A']
    
    # 不可分情况一： classList所有元素都相等，即类别完全相同，停止划分
    if classList.count('Iris-setosa') == len(classList):
        return 'Iris-setosa'
    if classList.count('Iris-versicolor') == len(classList):
        return 'Iris-versicolor'
    if classList.count('Iris-virginica') == len(classList):
        return 'Iris-virginica'
    
    # 不可分情况二： 已经遍历完所有特征时，返回出现次数最多的作为结果
    if len(labels) == 0:
        tmp_list = [classList.count('Iris-setosa'),classList.count('Iris-versicolor'),classList.count('Iris-virginica') ]
        name_list = ['Iris-setosa','Iris-versicolor','Iris-virginica']
        
        return name_list[tmp_list.index(max(tmp_list))]
        
    # 可分情况：划分
    return 1
        

In [1733]:
# 树的初始化与根节点

class Flower(object):  
    def __init__(self, name): 
        self.name = name

dictionary = {}
rootIndex,division_p  = choose_parent(train,4)                           #1-选出根节点
labels, rootNode = getPointName(rootIndex, 1, labels)                    #2-根节点的属性名字 len(labels)= 3  
dictionary[rootNode] = division_p
tree = Tree()                                                           #3-创建以rootNode为根节点的树  
tree.create_node(rootNode, rootNode)      # root
divide_judge = dividable(train, labels)                                 #4-判断是否可分
if divide_judge == 1 :         
    subset1, subset2 = update_train(train, rootIndex, division_p) 
else:               
    tree.create_node(divide_judge, identifier=None, parent=rootNode, data = Flower(divide_judge))   ß

In [1734]:
# 迭代子方法

def tmp_method(ts, parentNode, labels, tree):
    if ts != [] and len(labels) > 0:
        rootIndex,division_p  = choose_parent(ts,len(labels))                   #1-选出根节点
        labels, rootNode = getPointName(rootIndex, 2, labels)                   #2-根节点的属性名字 
        dictionary[rootNode] = division_p
        tree.create_node(rootNode,rootNode, parent=parentNode)                  #3-创建rootNode节点
        divide_judge = dividable(ts, labels)                                    #4-判断是否可分
        print(rootNode,' ', divide_judge)
        if divide_judge == 1:                #可分
            subset1, subset2 = update_train(ts, rootIndex, division_p)          #5-升级train
            tree = intera(subset1, subset2, parentNode, labels, tree)    #6-迭代,直到不可再分            
        else:                                #不可分，插入节点，结束这条链路
            #tree.create_node(divide_judge, identifier=None, parent=rootNode, data = Flower(divide_judge))
            tree.create_node(divide_judge, divide_judge, parent=rootNode)

    return tree

In [1735]:
# 8. 流程整合迭代

def intera(subset1, subset2, parentNode, labels, tree):
    tmp_set = [subset1, subset2]
    for ts in tmp_set:
        tree = tmp_method(ts, parentNode, labels, tree)
        
    return tree

In [1741]:
# 迭代测试
tree = intera(subset1, subset2, rootNode, labels, tree)
tree.show()

sepal_width
├── Iris-setosa
└── flower_length
    ├── flower_length
    │   ├── Iris-setosa
    │   └── Iris-versicolor
    └── flower_length
        ├── Iris-virginica
        └── flower_width
            ├── Iris-virginica
            └── flower_width
                ├── Iris-versicolor
                ├── flower_width
                │   └── Iris-versicolor
                └── flower_width
                    ├── Iris-virginica
                    └── flower_width
                        ├── Iris-versicolor
                        └── flower_width
                            ├── flower_length
                            │   ├── Iris-versicolor
                            │   ├── Iris-versicolor
                            │   ├── Iris-virginica
                            │   └── Iris-virginica
                            └── sepal_length



In [1743]:
# 6. 树的相关方法

# 定义树结构，初始化基本要素
class BinaryTree:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None
        
    # 插入左子树
    def insertLeft(self, newNode):
        if self.leftChild == None:                         #为空
            self.leftChild = BinaryTree(newNode)
        else:                                              #不为空
            t= BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    # 插入右子树
    def insertRight(self, newNode):
        if self.rightChild == None:                         #为空
            self.rightChild = BinaryTree(newNode)
        else:                                              #不为空
            t= BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t

    # 访问左子树
    def getLeftChild(self):
        return self.leftChild
    
    # 访问右子树
    def getRightChild(self):
        return self.rightChild
         
    # 访问该树的根节点
    def getRootVal(self):
        return self.key
